### Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación

### Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones 2022
Búsqueda y Recomendación para Textos Legales

Mentor: Jorge E. Pérez Villella

# Práctico Introducción al Aprendizaje Automático

Integrantes:
* Fernando Agustin Cardellino
* Adrian Zelaya

### Objetivos:

Probar distintos modelos de clasificación para evaluar la performance y la exactitud de predicción de cada modelo. 

* Utilizando el corpus normalizado en el práctico anterior, transformar el texto en vectores numéricos utilizando scikit-learn comparando los 3 modelos de vectorización. Explicar cada uno estos modelos.

* Clasificar los documentos por fuero. Trabajaremos con los siguientes modelos de clasificación de la librería scikit-learn: Logistic Regresion, Naive Bayes y SVM. En cada modelo probar distintos hiperparámetros, generar la Matriz de Confusión y la Curva ROC. Explicar los resultados obtenidos.

* Determinar y justificar cual es el modelo con mejor performance.

* Predecir el fuero de un documento utilizando el mejor modelo.

Opcional:

* Opcional: Profundizar el tema de stop words y cómo generar uno propio.

Fecha de Entrega: 29 de julio de 2022



In [2]:
# Importamos las librerías necesarias

import numpy as np
import pandas as pd
import os
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from wordcloud import WordCloud, ImageColorGenerator
from nltk.corpus import stopwords
from nltk.stem import porter, snowball
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
#import nltk
#nltk.download('stopwords')  # Descomentar para bajar las stopwords
# Ref: https://machinelearningmastery.com/prepare-text-data-machine-learning-scikit-learn/

In [3]:
# Cargamos modelo de spacy y stopwords de NLTK
nlp = spacy.load("es_core_news_sm")

CURR_DIR = os.getcwd()  # Gets current directory
STOPWORDS_ES = stopwords.words('spanish')
BREAKPOINT=5  # None para analizar todos los documentos, sino un número para analizar hasta n documentos
MAX_WORDS=1000
IMG_NAME = "legal-icon-png"


In [4]:
def getListOfFiles(dirName, return_dir=False):
    # create a list of file and sub directories
    # names in the given directory
    files = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for file in files:
        # Create full path
        fullPath = os.path.join(dirName, file)
        # If entry is a directory then get the list of files in this directory
        if os.path.isdir(fullPath):
            if return_dir:
                allFiles.append(fullPath.split(os.sep)[-1])
            else:
                allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)

    return allFiles

In [26]:
def get_tokens(files_path, fuero_name=None, breakpoint=None, object_type=None):
    """
    Función que arma el corpus de palabras y las tokeniza, utilizando SpaCy.
    Si `breakpoint` != None, dar el número de docs con el que se interrumpe la función
    object_type: indica el tipo de objeto que devuelve, i.e. 'token'/None (SpaCy object `token`), 'word' (texto/string) 
    or 'entity' (SpaCy object `entity`) 
    """
    def get_conditions(token, case):
        if case == 'entity':
            return True
        else:
            return token.is_alpha and not token.is_stop
    
    corpus = []#{} # lista de diccionarios

    tokens = [] # tokens de todo el documento

    i = 0
    for filename in getListOfFiles(files_path):
        tokens_ = [] #tokens por archivo
        file_name = filename.split(os.sep)[-1]
        fuero = fuero_name if fuero_name is not None else filename.split(os.sep)[-2]
        #if fuero not in corpus.keys():
        #    corpus[fuero] = {}

        with open(filename, encoding='utf-8') as file:
            file_text = file.read()
            # Tokenizamos el corpus
            nlp_doc = nlp(file_text)
            
            
            iterable = nlp_doc.ents if object_type == 'entity' else nlp_doc
            
            for token in iterable:
                if get_conditions(token, case=object_type):
                    aux_token = token.lemma_.lower() if object_type == 'word' else token
                    
                    tokens.append(# lematizamos y pasamos a minúscula (en modo 'palabras'/'word' solamente)
                        aux_token
                    )
                    
                    tokens_.append(# lematizamos y pasamos a minúscula (en modo 'palabras'/'word' solamente)
                        aux_token
                    )
            corpus.append({
                'id': i, 'archivo': file_name, 'fuero': fuero, 'texto': file_text, 'texto_clean': tokens_
            })
        
        i += 1            
        # remover esta sección (testing)
        if breakpoint:

            if i > breakpoint:
                break
    
    return tokens, pd.DataFrame(corpus)




def get_lemmas_stem_from_tokens(tokens, stemmer, language='spanish'):
    aux_dict = {
        'word': [token.lower_ for token in tokens],
        'lemma': [token.lemma_.lower() for token in tokens],
        'stem': [stemmer.stem(token.lower_) for token in tokens]
    }
    return pd.DataFrame(aux_dict)
    
    
def get_conteo_palabras(palabras):
    """Función que genera un pandas DataFrame con la frecuencia de las palabras
    """
    palabras_df = pd.DataFrame([{'palabra': str(x).lower()} for x in palabras])
    # print(corpus_df.head())

    return palabras_df.groupby(['palabra'])['palabra'].count().sort_values(ascending=False)

    
def generar_corpus(file_path, breakpoint, fuero_name=None, verbose=True, object_type='word'):
    if verbose:
        print("Generamos Corpus de palabras y conteo de frecuencias")
    
    # Generamos el corpus de palabras, y el diccionario con el mapeo de los fueros con sus respectivos documentos
    palabras, corpus = get_tokens(file_path, fuero_name=fuero_name, breakpoint=breakpoint, object_type='word')

    frecuencia_palabras_df = get_conteo_palabras(palabras)
    
    if verbose:
        print(f"Algunas palabras: {palabras[:5]}")
    
    return frecuencia_palabras_df, palabras, corpus


In [22]:
def vectorize_corpus(vectorizer, corpus):
    vectorizer_ = vectorizer(strip_accents='ascii', stop_words=STOPWORDS_ES)
    return vectorizer_.fit_transform(corpus)


In [6]:
# Ubicación de los documentos
filesDir = os.path.join(CURR_DIR, "Documentos")

# Obtenemos lista de los fueros
fueros = getListOfFiles(filesDir, return_dir=True)

In [27]:
n_docs = BREAKPOINT if not None else 'todos'
    
print(f"\nAnálisis para {n_docs} documentos\n")

frecuencia_palabras_df, corpus, corpus_df = generar_corpus(filesDir, BREAKPOINT, object_type='word', verbose=False)

print(frecuencia_palabras_df.head())

print(corpus)
    


Análisis para 5 documentos

palabra
art       80
luis      64
ley       62
vocal     62
doctor    54
Name: palabra, dtype: int64
['sala', 'laboral', 'tribunal', 'superior', 'protocolo', 'sentencias', 'nº', 'resolución', 'año', 'tomo', 'folio', 'despido', 'expediente', 'baez', 'pablo', 'adrian', 'flecha', 'bus', 'viajes', 'ordinario', 'sentencia', 'numero', 'cordoba', 'reúnir', 'integrante', 'sala', 'laboral', 'tribunal', 'superior', 'justicia', 'doctor', 'luis', 'enrique', 'rubio', 'mercedes', 'blanc', 'arabel', 'luis', 'eugenio', 'angulo', 'presidencia', 'nombrado', 'dictar', 'sentencia', 'auto', 'baez', 'pablo', 'adrian', 'flecha', 'bus', 'viajes', 'ordinario', 'despido', 'recurso', 'casacion', 'raíz', 'recurso', 'concedido', 'actoro', 'sentencia', 'n', 'dictado', 'sala', 'novena', 'cámara', 'única', 'trabajo', 'constituido', 'tribunal', 'unipersonal', 'cargo', 'señor', 'juez', 'doctor', 'pedro', 'antonio', 'grasso', 'n', 'cuyo', 'copia', 'obra', 'fs', 'vta', 'resolver', 'rechazar',

In [28]:
print(corpus)
print(len(corpus))

['sala', 'laboral', 'tribunal', 'superior', 'protocolo', 'sentencias', 'nº', 'resolución', 'año', 'tomo', 'folio', 'despido', 'expediente', 'baez', 'pablo', 'adrian', 'flecha', 'bus', 'viajes', 'ordinario', 'sentencia', 'numero', 'cordoba', 'reúnir', 'integrante', 'sala', 'laboral', 'tribunal', 'superior', 'justicia', 'doctor', 'luis', 'enrique', 'rubio', 'mercedes', 'blanc', 'arabel', 'luis', 'eugenio', 'angulo', 'presidencia', 'nombrado', 'dictar', 'sentencia', 'auto', 'baez', 'pablo', 'adrian', 'flecha', 'bus', 'viajes', 'ordinario', 'despido', 'recurso', 'casacion', 'raíz', 'recurso', 'concedido', 'actoro', 'sentencia', 'n', 'dictado', 'sala', 'novena', 'cámara', 'única', 'trabajo', 'constituido', 'tribunal', 'unipersonal', 'cargo', 'señor', 'juez', 'doctor', 'pedro', 'antonio', 'grasso', 'n', 'cuyo', 'copia', 'obra', 'fs', 'vta', 'resolver', 'rechazar', 'parte', 'demanda', 'incoado', 'pablo', 'adrian', 'baez', 'dni', 'flecha', 'bus', 'viajes', 'costa', 'diferir', 'regulación', 'ho

Utilizando el corpus normalizado en el práctico anterior, transformar el texto en vectores numéricos utilizando scikit-learn comparando los 3 modelos de vectorización. Explicar cada uno estos modelos.

# CountVectorizer
(acá iría descripción de qué es)


In [23]:
# CountVectorizer
# Ver warning: https://stackoverflow.com/questions/57340142/user-warning-your-stop-words-may-be-inconsistent-with-your-preprocessing
#vectorizer_1 = CountVectorizer(strip_accents='ascii', stop_words=STOPWORDS_ES) # Both ‘ascii’ and ‘unicode’ use NFKD normalization from unicodedata.normalize.
#X_1 = vectorizer_1.fit_transform(corpus) # Learn the vocabulary dictionary and return document-term matrix.

X_1 = vectorize_corpus(CountVectorizer, corpus)
#print(vectorizer_1.get_feature_names_out())
print(X_1.toarray())
print(X_1.toarray().shape)


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(4920, 1354)


# TfidfVectorizer
(descripción)

In [24]:
# ver normas L1 o L2 (default: L2)
# ver de pasar los documentos y configurar los parámetros de preprocesamiento
X_2 = vectorize_corpus(TfidfVectorizer, corpus)

X_train 
X_train_vectorized = [X_1, X_2, .. , X_n] 
[X_1 + X_2 + ...]

print(X_2.toarray())
print(X_2.toarray().shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(4920, 1354)


# HashingVectorizer

In [25]:
# además de norm, ver n_features, alternate_sign
# ver de pasar los documentos y configurar los parámetros de preprocesamiento
X_3 = vectorize_corpus(HashingVectorizer, corpus)
print(X_3)
#print(vectorizer_3.get_feature_names_out())
#print(X_3.toarray())
#print(X_3.toarray().shape)

  (0, 875449)	1.0
  (1, 264889)	1.0
  (2, 901816)	1.0
  (3, 1030371)	1.0
  (4, 418843)	-1.0
  (5, 464956)	1.0
  (7, 342478)	-1.0
  (8, 168320)	-1.0
  (9, 307731)	-1.0
  (10, 594346)	-1.0
  (11, 69024)	1.0
  (12, 48516)	-1.0
  (13, 758632)	-1.0
  (14, 513511)	1.0
  (15, 852111)	1.0
  (16, 565662)	-1.0
  (17, 763499)	-1.0
  (18, 889640)	1.0
  (19, 440395)	1.0
  (20, 932)	-1.0
  (21, 686445)	-1.0
  (22, 675406)	1.0
  (23, 853864)	-1.0
  (24, 927066)	-1.0
  (25, 875449)	1.0
  :	:
  (4895, 873084)	1.0
  (4896, 102505)	1.0
  (4897, 33509)	1.0
  (4898, 247529)	1.0
  (4899, 463429)	-1.0
  (4900, 363968)	-1.0
  (4901, 551155)	-1.0
  (4902, 869843)	1.0
  (4903, 79494)	-1.0
  (4904, 691029)	-1.0
  (4905, 894734)	-1.0
  (4906, 898506)	-1.0
  (4907, 951425)	-1.0
  (4908, 815788)	1.0
  (4909, 21045)	1.0
  (4910, 397778)	1.0
  (4911, 176801)	1.0
  (4912, 869843)	1.0
  (4913, 511454)	-1.0
  (4914, 981843)	1.0
  (4915, 924629)	-1.0
  (4916, 846701)	1.0
  (4917, 527938)	-1.0
  (4918, 48516)	-1.0
  (4919

In [36]:
### REVISAR !!!!!!
# Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

def simple_logistic_classify(X_tr, y_tr, X_test, y_test, description, _C=1.0):
    model = LogisticRegression(C=_C).fit(X_tr, y_tr)
    score = model.score(X_test, y_test)
    print('Test Score with', description, 'features', score)
    return model


X = corpus_df[[x for x in corpus_df.columns if x != 'fuero']]
y = corpus_df['fuero']


X_train, y_train, X_test, y_test = train_test_split(X, y, test_size=0.3)




    

,id,archivo,texto,texto_clean
4,4,188 LUCIANO-NICOLAS.pdf.txt,SALA LABORAL - TRIBUNAL SUPERIOR\n\nProtocolo ...,"[sala, laboral, tribunal, superior, protocolo,..."
0,0,9 BAEZ-FLECHA BUS.pdf.txt,SALA LABORAL - TRIBUNAL SUPERIOR\n\nProtocolo ...,"[sala, laboral, tribunal, superior, protocolo,..."
5,5,220 MURUA-PROVINCIA.pdf.txt,SALA LABORAL - TRIBUNAL SUPERIOR\n\nProtocolo ...,"[sala, laboral, tribunal, superior, protocolo,..."
3,3,3 SANGUEDOLCE-MUNICIPALIDAD DE VILLA ALLENDE.p...,SALA LABORAL - TRIBUNAL SUPERIOR\n\nProtocolo ...,"[sala, laboral, tribunal, superior, protocolo,..."
